## Dashboard for displaying visualizations

In [5]:
import numpy as np
import pandas as pd
import altair as alt
from dash import dash, dcc, html, Input, Output
from altair import datum
import os
import json

In [6]:
nav_events = pd.read_csv('hackathon_alternate_data/navigation_events.csv')
disc = pd.read_csv('hackathon_alternate_data/additional/discussions.csv')
disc_dates = pd.read_csv('hackathon_alternate_data/additional/discussion_topics.csv')

In [7]:
disc['timestamp'] = pd.to_datetime(disc['timestamp'])
disc['date'] = disc['timestamp'].dt.normalize()#date
disc_merged = disc.merge(disc_dates[['title','todo_date_date']],how='inner',left_on='discussion_topic_title',right_on='title')

In [8]:
# Plot hist of assignment submission dates
def plot_altair(discussion_topic):
    chart = alt.Chart(disc_merged).mark_bar().encode(
        alt.X('date:T').title("Date"),
        alt.Y('count(x):N').title("Number of Submissions")).transform_filter(datum.discussion_topic_title == discussion_topic)
    line = chart.mark_rule(color='red').encode(x = 'todo_date_date:T')
    return (chart+line).to_html()

In [54]:
# Simon's cumulative assignment view viz + data cleaning
assignments = pd.read_csv("hackathon_alternate_data/additional/assignments.csv")
navigation_events = pd.read_csv("hackathon_alternate_data/navigation_events.csv")
enrollments = pd.read_csv("hackathon_alternate_data/additional/enrollments.csv")

# drop strange/na columns
navigation_events.drop(columns=["ed_app", "type", "action", "course_offering_id", "statement_type", "statement_version", "event__object_type", "event__object_extensions_asset_subtype", "event__object_extensions_entity_id", "event__referrer", "event__extensions_request_url", "event__attachment_type"], inplace=True)
# drop learners that are not enrolled in the course
navigation_events = navigation_events.iloc[np.in1d(navigation_events.actor_id, enrollments.user_id), :]
navigation_events = navigation_events.query("actor_id != 'LEARNER_48'")
navigation_events.loc[:, "event_time"] = pd.to_datetime(navigation_events.loc[:, "event_time"], format='ISO8601').dt.normalize()

df_assignments = assignments.drop(columns=["unlock_at", "lock_at", "grading_type", "position"]).query("has_submitted_submissions == True")
df_assignments["due_at"] = pd.to_datetime(df_assignments["due_at"])

def plot_cumulative_assignment_view(assignment_name):
    
    ## Drawing
    tmp_assignment = df_assignments.query("name == @assignment_name")
    tmp_assignment_due_date = tmp_assignment.due_at
    tmp_first_accessed = navigation_events.query("object_id == @tmp_assignment.id.iloc[0]")[["actor_id", "event_time"]]
    tmp_first_accessed = tmp_first_accessed.sort_values(["actor_id", "event_time"]).groupby("actor_id").first().sort_values("event_time")
    
    tmp_first_accessed = tmp_first_accessed.reset_index()
    tmp_first_accessed['event_time'] = pd.to_datetime(tmp_first_accessed['event_time'])
    chart = alt.Chart(tmp_first_accessed).transform_window(
        ecdf="cume_dist()",
        sort=[{"field": "event_time"}],
    ).mark_line().encode( #interpolate="step-after"
        x=alt.X("event_time:T").scale(domain=[tmp_first_accessed.event_time.iloc[0] - pd.Timedelta(days=7),  tmp_first_accessed.event_time.iloc[-1] + pd.Timedelta(days=7)]),
        y=alt.Y("ecdf:Q").scale(domain=[0, 1.1]).title("Assignment View %"), 
    )
    chart = chart + alt.Chart(tmp_assignment).mark_rule().encode(x = alt.X("due_at").title(None))
    return chart.properties(width=600, height=300).to_html()

#plot_cumulative_assignment_view(Assignment 1")

NameError: name 'gradebook' is not defined

In [58]:
gradebook = pd.read_csv(os.path.join("hackathon_alternate_data", "additional", "gradebook.csv"))
gradebook.drop(index=[0,1], inplace=True)
gradebook.loc[:, gradebook.columns != "Student"] = gradebook.loc[:, gradebook.columns != "Student"].apply(pd.to_numeric)

,Student,Assignment 1 (c81f04547a95da2a7b88054ef491b7c4),Assignment 2 (a4dc11e7e79361fc5886a9078aac66b8),Assignment 3 (option A) (811d93ea379b5cdd5a19f1b5dbab88cd),Assignment 3 (option B) (df8d1f1ff3f48fdc24a278b40c5f45cc),Assignment 3 (option C) (bb971b36c1578cede00150acda89aa99),Assignment 3 (option D) (f1ad954cd2cddda6e17f6fc225d1aa3e),Participation grade: Formative (not included in final grade) (5965e0b0c712861d0efdd9be54572114),Assignment 1 Current Score,Assignment 2 Current Score,Assignment 3 Current Score,Participation & engagement Current Score,Current Score
2,LEARNER_4,88,76.0,75.0,NaN,NaN,NaN,10.0,88,76.0,75.0,100.0,82.9
3,LEARNER_10,88,72.0,74.0,NaN,NaN,NaN,8.5,88,72.0,74.0,85.0,78.4
4,LEARNER_34,90,78.0,73.0,NaN,NaN,NaN,9.0,90,78.0,73.0,90.0,81.3
5,LEARNER_16,88,71.0,NaN,NaN,77.0,NaN,10.0,88,71.0,77.0,100.0,82.0
6,LEARNER_7,92,83.0,NaN,NaN,80.0,NaN,10.0,92,83.0,80.0,100.0,87.3
7,LEARNER_30,77,89.0,NaN,NaN,74.0,NaN,7.0,77,89.0,74.0,70.0,78.3
8,LEARNER_15,90,79.0,75.0,NaN,NaN,NaN,6.0,90,79.0,75.0,60.0,76.2
9,LEARNER_19,90,83.0,NaN,NaN,81.0,NaN,9.0,90,83.0,81.0,90.0,85.2
10,LEARNER_36,56,84.0,75.0,NaN,NaN,NaN,6.0,56,84.0,75.0,60.0,70.9
11,LEARNER_8,90,88.0,74.0,NaN,NaN,NaN,8.5,90,88.0,74.0,85.0,83.6


In [77]:
assignments = pd.read_csv("hackathon_alternate_data/additional/assignments.csv")
navigation_events = pd.read_csv("hackathon_alternate_data/navigation_events.csv")
enrollments = pd.read_csv("hackathon_alternate_data/additional/enrollments.csv")

# drop strange/na columns
navigation_events.drop(columns=["ed_app", "type", "action", "course_offering_id", "statement_type", "statement_version", "event__object_type", "event__object_extensions_asset_subtype", "event__object_extensions_entity_id", "event__referrer", "event__extensions_request_url", "event__attachment_type"], inplace=True)
# drop learners that are not enrolled in the course
navigation_events = navigation_events.iloc[np.in1d(navigation_events.actor_id, enrollments.user_id), :]
navigation_events = navigation_events.query("actor_id != 'LEARNER_48'")
navigation_events.loc[:, "event_time"] = pd.to_datetime(navigation_events.loc[:, "event_time"], format='ISO8601').dt.normalize()

def plot_daily_canvas_interactions(student): 
    # preprocess
    df_count = pd.DataFrame({"count" : navigation_events.query("actor_id == @student").groupby("event_time").count().id})
    a = pd.DataFrame(pd.date_range(df_count.index.min(), df_count.index.max(), freq="D"), columns=["date"])
    a = pd.merge(a, df_count, how="left", left_on="date", right_index=True)
    a = a.fillna(0)
    a["weekday"]=a.date.dt.strftime("%a")
    a["week"]=a.date.dt.isocalendar().week

    # draw
    chart = alt.Chart(a, title="Daily Canvas Interactions").mark_rect().encode(
        x = alt.X("week(date):O").title(None),
        y = alt.Y("day(date):O").title(None),
        color = alt.Color("count:Q").title(None), 
        tooltip = ["date", "count"]
    )

    return chart.properties(width=600, height=300).to_html()

gradebook = pd.read_csv(os.path.join("hackathon_alternate_data", "additional", "gradebook.csv"))
gradebook.drop(index=[0,1], inplace=True)
gradebook.loc[:, gradebook.columns != "Student"] = gradebook.loc[:, gradebook.columns != "Student"].apply(pd.to_numeric)

def plot_grades_as_table(student):
    table_out = gradebook.query("Student == @student").T.dropna().drop("Student")
    table_out.columns = [student]
    return table_out.to_html()


In [75]:
# Set up dashboard
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

assignments = ['Assignment 1', 'Assignment 2', 'Assignment 3 (option A)', 
               'Assignment 3 (option B)','Assignment 3 (option C)','Assignment 3 (option D)']
discussion_topics = disc['discussion_topic_title'].unique()

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Home', children=[html.H1('Course-123 Instructor Analytics'),
            html.P('Welcome to your instructor dashboard! Navigate through the tabs for insights on student engagement and performance'),
            dcc.Graph(
                figure={
                    'data': [
                        {'x': [1, 2, 3], 'y': [2, 4, 3],
                            'type': 'bar', 'name': 'SF'},
                        {'x': [1, 2, 3], 'y': [5, 4, 3],
                         'type': 'bar', 'name': 'Montréal'},
                    ]
                }
            )]),
        
        dcc.Tab(label='Discussion Posts', children=[
            dcc.Dropdown(
            id='discussion', value='Discussion 1: The meaning of eLearning',
            options=[{'label': i, 'value': i} for i in discussion_topics]),
        html.Iframe(
            id='disc-scatter',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_altair('Discussion 1: The meaning of eLearning'))]),
        
        dcc.Tab(label='Assignments', children=[
            dcc.Dropdown(
            id='assignment_name', value='Assignment 1',
            options=[{'label': i, 'value': i} for i in assignments]),
        html.Iframe(
            id='a-scatter',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_cumulative_assignment_view('Assignment 1'))
        ]),
        
        dcc.Tab(label='Individual Students', children=[
            dcc.Dropdown(
            id='student_id', value='LEARNER_1',
            options=[{'label': i, 'value': i} for i in navigation_events.actor_id.unique()]),
        html.Iframe(
            id='daily_canvas_interactions',
            style={'border-width': '0', 'width': '100%', 'height': '400px'},
            srcDoc=plot_daily_canvas_interactions("LEARNER_1")),
        html.Iframe(
            id='grades',
            style={'border-width': '0', 'width': '75%', 'height': '500px'},
            srcDoc=plot_cumulative_assignment_view('Assignment 1'))
        ]),
    ])    
])

# Callback for updating the 'Discussion'
@app.callback(
    Output('disc-scatter', 'srcDoc'),
    Input('discussion', 'value')
)
def update_disc_scatter(discussion_topic):
    return plot_altair(discussion_topic)

# Callback for updating the 'Assignments'
@app.callback(
    Output('a-scatter', 'srcDoc'),
    Input('assignment_name', 'value')
)
def update_a_scatter(assignment_name):
    return plot_cumulative_assignment_view(assignment_name)

# Callback for updating "Individual Students"
@app.callback(
    Output('daily_canvas_interactions', 'srcDoc'),
    Input('student_id', 'value')
)
def update_daily_canvas_interactions(student):
    return plot_daily_canvas_interactions(student)
# Callback for updating "Individual Students" (table)
@app.callback(
    Output('grades', 'srcDoc'),
    Input('student_id', 'value')
)
def update_grades(student):
    return plot_grades_as_table(student)



if __name__ == '__main__':
    app.run_server(debug=True)